# Train a Text Generation  model using LSTM and TensorFlow

Welcome to this notebook on text generation using LSTM and TensorFlow. This notebook offers a straightforward approach to understanding and implementing a simple LSTM-based text generation model.

In [ ]:
from datasets import load_dataset
import numpy as np
import pandas as pd 
from collections import Counter
import matplotlib.pyplot as plt      
from sklearn.model_selection import train_test_split
import random
import re                                  
import string         
import nltk   
from nltk.tokenize import word_tokenize                     
from nltk.corpus import stopwords  
from nltk.tokenize import sent_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
from keras.layers import Attention
from tensorflow.keras.models import load_model


In [ ]:
# load the dataset 
dataset = load_dataset("bookcorpus",split='train[:50%]') 
dataset

In [ ]:
ds=dataset.to_pandas()
ds=ds['text']